In [1]:
import requests
import numpy as np
!pip install "tensorflow>=1.14.0,<2.0"
!pip install "tensorflow-gpu>=1.14.0,<2.0"
!pip install  ampligraph


In [6]:
import ampligraph
import numpy as np
ampligraph.__version__
import pandas as pd
from ampligraph.datasets import load_from_ntriples
from ampligraph.datasets import load_from_csv

In [9]:
Y=load_from_csv('/content','1000data.csv',sep=',',add_reciprocal_rels=False)

In [10]:
print(Y)

[['Common symptoms' 'include' 'fever']
 ['Management' 'involves' 'treatment']
 ['The World Health Organization' 'declared' 'outbreak']
 ...
 ['black people' 'resisted' 'slavery']
 ['Leading Black activists' 'included' 'Olaudah Equiano']
 ['Supporters' 'included' 'workers']]


In [11]:
Y.shape

(978, 3)

In [12]:
entities = np.unique(np.concatenate([Y[:, 0], Y[:, 2]]))
entities.size

1626

In [13]:
relations = np.unique(Y[:, 1])
relations.size

448

In [14]:
from ampligraph.evaluation import train_test_split_no_unseen 

In [16]:
from ampligraph.evaluation import train_test_split_no_unseen 

Y_train, Y_test = train_test_split_no_unseen(Y, test_size=25) 

In [17]:
print('Train set size: ', Y_train.shape)
print('Test set size: ', Y_test.shape)

Train set size:  (953, 3)
Test set size:  (25, 3)


In [18]:
print(Y_train[:200])

[['Common symptoms' 'include' 'fever']
 ['Management' 'involves' 'treatment']
 ['The World Health Organization' 'declared' 'outbreak']
 ['common symptoms' 'include' 'cough']
 ['minority' 'develop' 'noticeable symptoms']
 ['Cardiovascular complications' 'include' 'heart failure']
 ['Sputum' 'carry' 'large amounts']
 ['indicative' 'suggest' 'underlying immunopathology']
 ['people' 'have' 'classical serum biomarkers']
 ['The US Food and Drug Administration' 'approved' 'test']
 ['guidelines' 'recommend' 'medication']
 ['Face coverings' 'limit' 'volume']
 ['the United States Environmental Protection Agency' 'maintains' 'list']
 ['Intensivists' 'compiled' 'treatment recommendations']
 ['availability' 'affect' 'mortality']
 ['study' 'reported' 'earliest date']
 ['Official publications' 'reported' 'earliest onset']
 ['Wuhan Central Hospital' 'sent'
  'bronchoalveolar lavage fluid BAL sample']
 ['the National Health Commission of China' 'issued' 'notice']
 ['China' 'reported' 'nearly 140 new ca

In [19]:
from ampligraph.latent_features import DistMult

In [23]:
model = DistMult(batches_count=10, 
                seed=0, 
                epochs=20, 
                k=10, 
                eta=10,
                optimizer='adam', 
                optimizer_params={'lr':1e-3},
                loss='multiclass_nll', 
                regularizer='LP', 
                regularizer_params={'p':3, 'lambda':1e-5}, 
                verbose=True)

In [ ]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

model.fit(Y_train, early_stopping = False)

In [29]:
model.get_embeddings(['The World Health Organization','outbreak'], embedding_type='entity')

array([[-0.14007236, -0.17432842,  0.08273704, -0.16811879,  0.17377837,
         0.08157767, -0.08046551,  0.19293568, -0.04374849,  0.10789777],
       [-0.13622098,  0.10265293, -0.06040851,  0.11214289, -0.10017645,
        -0.11486229,  0.05497329, -0.10594554, -0.02721807, -0.0632102 ]],
      dtype=float32)

In [30]:
model.get_embeddings(['common symptoms','cough'], embedding_type='entity')

array([[-0.08037735,  0.16119793,  0.05594945,  0.11175679, -0.14262652,
         0.11011828,  0.11908375, -0.01564922, -0.0615891 , -0.16319677],
       [ 0.06842158, -0.09555812, -0.00932129, -0.08976121,  0.10881982,
         0.07787556,  0.0649714 ,  0.01940289, -0.03769498, -0.1358584 ]],
      dtype=float32)

In [31]:
positives_filter = Y

In [32]:
from ampligraph.latent_features import save_model, restore_model

In [33]:
save_model(model, './covid_model.pkl')

In [34]:
from ampligraph.evaluation import evaluate_performance

In [35]:
from ampligraph.discovery import find_duplicates

In [36]:
entities = np.unique(Y_train[:, 2])
dups, _ = find_duplicates(entities, model, mode='entity', tolerance=0.4)
print(list(dups)[:3])

[frozenset({'Canada', 'YouTube educational video', 'species', 'trials', 'the World Health Organization', 'preference', 'the Mississippi River', 'impact', 'body immune system', 'between 10 and 33%', 'viral myocarditis long QT syndrome Brugada syndrome', 'subsidy', 'suppression', 'best results', 'relatively large exposure', 'continuous population', 'well over 80%', 'minor continuing presence', 'order', 'nasal polyps', 'fast deadly arrhythmias', 'million', 'national IFR', '1TB', 'average', 'biosimilar', 'rate', 'Disobey Idaho', 'mental health', 'developed neutralizing antibodies', 'parasympathetic nervous supply', '12%', 'inner membrane', 'patient care', 'bacteria', 'egg', 'motor vehicles', 'stops', 'earliest date', 'roles', 'ABO hemolytic disease', 'S protein', 'bat houses', 'ArcGIS Mobile Desktop software', '50,000', 'major roles', 'pelvic inflammatory disease', 'endosome', 'significant role', '65%', 'Esri software development', '620 points', 'tissue injury', 'about 2% to 3%', 'five tim

In [37]:
from ampligraph.discovery import query_topn

In [38]:
query_topn(model, top_n=5,
            head=None, relation='include', tail='cough',
           ents_to_consider=None, rels_to_consider=None)

(array([['common symptoms', 'include', 'cough'],
        ['orders', 'include', 'cough'],
        ['62.5 per cent', 'include', 'cough'],
        ['ARDS', 'include', 'cough'],
        ['rest', 'include', 'cough']], dtype='<U70'),
 array([0.02639998, 0.02377397, 0.01937441, 0.01922832, 0.0188348 ],
       dtype=float32))

In [39]:
ranks = evaluate_performance(Y_test, 
                             model=model, 
                             filter_triples=positives_filter,   # Corruption strategy filter defined above 
                             use_default_protocol=True, # corrupt subj and obj separately while evaluating
                             verbose=True)

WARNING - DeprecationWarning: use_default_protocol will be removed in future. Please use corrupt_side argument instead.


100%|██████████| 25/25 [00:00<00:00, 297.03it/s]


In [68]:
from ampligraph.evaluation import mr_score, mrr_score, hits_at_n_score
mr = mr_score(ranks)
mrr = mrr_score(ranks)
print("MRR: %.2f" % (mrr))
print("MR: %.2f" % (mr))
hits_10 = hits_at_n_score(ranks, n=10)
print("Hits@10: %.2f" % (hits_10))
hits_3 = hits_at_n_score(ranks, n=3)
print("Hits@3: %.2f" % (hits_3))
hits_1 = hits_at_n_score(ranks, n=1)
print("Hits@1: %.2f" % (hits_1))

MRR: 0.03
MR: 747.12
Hits@10: 0.04
Hits@3: 0.04
Hits@1: 0.02


In [41]:
from ampligraph.utils import create_tensorboard_visualizations

In [42]:
create_tensorboard_visualizations(model,r"./")

In [62]:
X_unseen = np.array([
    ['guidelines', 'recommend', 'medication'],
  ['black musician', 'wears', 'royal livery'],
   ['Common effects', 'include', 'chest infections'],
 #   ['pandemic', 'include', 'global social economic disruption']

])

In [63]:
unseen_filter = np.array(list({tuple(i) for i in np.vstack((positives_filter, X_unseen))}))

In [64]:
ranks_unseen = evaluate_performance(
    X_unseen, 
    model=model, 
    filter_triples=unseen_filter,   # Corruption strategy filter defined above 
    corrupt_side = 's+o',
    use_default_protocol=False, # corrupt subj and obj separately while evaluating
    verbose=True
)

100%|██████████| 3/3 [00:00<00:00, 59.77it/s]


In [65]:
scores = model.predict(X_unseen)

In [66]:
from scipy.special import expit
probs = expit(scores)

In [67]:
pd.DataFrame(list(zip([' '.join(x) for x in X_unseen], 
                      ranks_unseen, 
                      np.squeeze(scores),
                      np.squeeze(probs))), 
             columns=['statement', 'rank', 'score', 'prob']).sort_values("score")

,statement,rank,score,prob
2,Common effects include chest infections,795,0.006087,0.501522
1,black musician wears royal livery,6,0.010364,0.502591
0,guidelines recommend medication,2,0.014578,0.503644
